In [187]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import sys

In [188]:
tf.disable_v2_behavior()

# import data and split to train/test , x/y.

In [189]:
#there are 24 letters but for the order j get an index
num_categories=25 

# Importing the data - test and train
test = pd.read_csv("sign_mnist_test.csv").to_numpy() 
train = pd.read_csv("sign_mnist_train.csv").to_numpy()

# Splitting test into x & y - x = fetures y = labels 
# Note: we do OneHot on the y to get it as a vector at the size of the categories
# then put's 1 where the right label is and zero in the rest
y_test=test[:,0]
y_test=keras.utils.to_categorical(y_test, num_categories)
x_test=test[:,1:]

# Splitting data into x & y - x = fetures y = labels
# Note: we do OneHot on the y to get it as a vector at the size of the categories
# then put's 1 where the right label is and zero in the rest
data_y=train[:,0]
data_y=keras.utils.to_categorical(data_y, num_categories)
data_x=train[:,1:]

# Getting the number of feturs
# Note: in the case of image each pixel and the toal number is n*n
lines,features=data_x.shape

biggestClass=np.argmax(np.bincount(train[:,0]))
precentOfDataOfBiggestClass=np.max(np.bincount(train[:,0]))*100/lines

print("there are 24 classes,the class that has the most data is class %2d with %.3f precents of the data"
      %(biggestClass,precentOfDataOfBiggestClass))
print("if the data were equals between all classes it needed to be %.3f precents of the data" %(100/24))

there are 24 classes,the class that has the most data is class 17 with 4.713 precents of the data
if the data were equals between all classes it needed to be 4.167 precents of the data


In [190]:
# A "list like" object that will hold the calculatins as tensors
# Size of x: (?, 784)
x = tf.placeholder(tf.float32, [None, features])
# Size of y_: (?, 25)
y_ = tf.placeholder(tf.float32, [None, num_categories])


W = tf.Variable(tf.zeros([features,num_categories]))
b = tf.Variable(tf.zeros([num_categories]))

# Define activation function - Softmax 
y = tf.nn.softmax(tf.matmul(x, W) + b)

# Define the loss function - Cross entropy
loss = -tf.reduce_mean(y_*tf.log(y))

# Define the update function - Gradient descent
update = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

# Define accuracy - Percentage of predictions did we get right
correct_prediction = tf.equal(tf.argmax(y, 1),tf.argmax(y_, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) 

In [191]:
print(data_x.shape)
print(data_y.shape)

(27455, 784)
(27455, 25)


In [192]:
print(x.shape)
print(y_.shape)

(?, 784)
(?, 25)


In [193]:
tf.print(W, output_stream=sys.stdout)

<tf.Operation 'PrintV2_14' type=PrintV2>

# Logistic Regression

In [195]:
# Define a session
sess = tf.Session()
# Init all the vaeiables
sess.run(tf.global_variables_initializer())

for epoch in range(0,50):
    sess.run(update, feed_dict = {x:data_x, y_:data_y}) #BGD 
    train_acc = sess.run(accuracy, feed_dict={x:data_x, y_:data_y})
    test_acc = sess.run(accuracy, feed_dict={x:x_test, y_:y_test})
    print ("epoch: %3d train_acc: %.3f test_acc: %.3f" % (epoch,train_acc, test_acc))


epoch:   0 train_acc: 0.047 test_acc: 0.020
epoch:   1 train_acc: 0.190 test_acc: 0.253
epoch:   2 train_acc: 0.047 test_acc: 0.020
epoch:   3 train_acc: 0.190 test_acc: 0.199
epoch:   4 train_acc: 0.087 test_acc: 0.051
epoch:   5 train_acc: 0.125 test_acc: 0.094
epoch:   6 train_acc: 0.087 test_acc: 0.050
epoch:   7 train_acc: 0.144 test_acc: 0.130
epoch:   8 train_acc: 0.084 test_acc: 0.061
epoch:   9 train_acc: 0.182 test_acc: 0.179
epoch:  10 train_acc: 0.106 test_acc: 0.086
epoch:  11 train_acc: 0.090 test_acc: 0.086
epoch:  12 train_acc: 0.111 test_acc: 0.048
epoch:  13 train_acc: 0.143 test_acc: 0.135
epoch:  14 train_acc: 0.118 test_acc: 0.116
epoch:  15 train_acc: 0.223 test_acc: 0.184
epoch:  16 train_acc: 0.102 test_acc: 0.078
epoch:  17 train_acc: 0.166 test_acc: 0.152
epoch:  18 train_acc: 0.139 test_acc: 0.103
epoch:  19 train_acc: 0.157 test_acc: 0.109
epoch:  20 train_acc: 0.228 test_acc: 0.187
epoch:  21 train_acc: 0.137 test_acc: 0.152
epoch:  22 train_acc: 0.198 test

# Simple NN

In [196]:
(hidden1_size, hidden2_size) = (100, 50)

# Set the weight of the first layer - 784*100 
# The values to be normalized [0.1,1]
W1 = tf.Variable(tf.truncated_normal([features, hidden1_size], stddev=0.1))

# Set the bais of the first layer - 100*1
# The values - 0.1
b1 = tf.Variable(tf.constant(0.1, shape=[hidden1_size]))

# Set the output of the first layer using ReLU
# Output size: [?,100]
z1 = tf.nn.relu(tf.matmul(x,W1)+b1)

# Set the weight of the second layer - 100*50 
# The values to be normalized [0.1,1]
W2 = tf.Variable(tf.truncated_normal([hidden1_size, hidden2_size], stddev=0.1))

# Set the bais of the second layer - 50*1
# The values - 0.1
b2 = tf.Variable(tf.constant(0.1, shape=[hidden2_size]))

# Set the output of the second layer using ReLU
# Output size: [?,50]
z2 = tf.nn.relu(tf.matmul(z1,W2)+b2)

# Set the weight of the output layer - 50*25
# The values to be normalized [0.1,1]
W3 = tf.Variable(tf.truncated_normal([hidden2_size, num_categories], stddev=0.1))

# Set the bais of the output layer - 25*1
# The values - 0.1
b3 = tf.Variable(tf.constant(0.1, shape=[num_categories]))

# Set the output using SoftMax
# Output size: [?,25]
y = tf.nn.softmax(tf.matmul(z2, W3) + b3)

# Define the loss function - Cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# Define the update function - Gradient descent
update = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)


In [205]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(0,1001):
    sess.run(update, feed_dict = {x:data_x, y_:data_y})
    if epoch%10 == 0:
        train_acc = sess.run(accuracy, feed_dict={x:data_x, y_:data_y})
        test_acc = sess.run(accuracy, feed_dict={x:x_test, y_:y_test})
        print ("epoch: %3d train_acc: %.3f test_acc: %.3f" % (epoch,train_acc, test_acc))

epoch:   0 train_acc: 0.047 test_acc: 0.020
epoch:  10 train_acc: 0.109 test_acc: 0.090
epoch:  20 train_acc: 0.142 test_acc: 0.147
epoch:  30 train_acc: 0.232 test_acc: 0.185
epoch:  40 train_acc: 0.275 test_acc: 0.261
epoch:  50 train_acc: 0.298 test_acc: 0.254
epoch:  60 train_acc: 0.297 test_acc: 0.229
epoch:  70 train_acc: 0.310 test_acc: 0.256
epoch:  80 train_acc: 0.330 test_acc: 0.270
epoch:  90 train_acc: 0.345 test_acc: 0.283
epoch: 100 train_acc: 0.360 test_acc: 0.312
epoch: 110 train_acc: 0.372 test_acc: 0.319
epoch: 120 train_acc: 0.395 test_acc: 0.342
epoch: 130 train_acc: 0.403 test_acc: 0.339
epoch: 140 train_acc: 0.421 test_acc: 0.358
epoch: 150 train_acc: 0.425 test_acc: 0.358
epoch: 160 train_acc: 0.440 test_acc: 0.370
epoch: 170 train_acc: 0.446 test_acc: 0.377
epoch: 180 train_acc: 0.455 test_acc: 0.381
epoch: 190 train_acc: 0.457 test_acc: 0.375
epoch: 200 train_acc: 0.463 test_acc: 0.384
epoch: 210 train_acc: 0.469 test_acc: 0.388
epoch: 220 train_acc: 0.475 test

In [198]:
(hidden1_size, hidden2_size) = (100, 50)

# Set the weight of the first layer - 784*100 
# The values to be normalized [0.1,1]
W1 = tf.Variable(0.1 * np.random.randn(784, hidden1_size).astype(np.float32))

# Set the bais of the first layer - 100*1
# The values - 0.1
b1 = tf.Variable(tf.constant(0.1, shape=[hidden1_size]))

# Set the output of the first layer using ReLU
# Output size: [?,100]
z1 = tf.nn.relu(tf.matmul(x,W1)+b1)

# Set the weight of the second layer - 100*50 
# The values to be normalized [0.1,1]
W2 = tf.Variable(0.1 * np.random.randn(hidden1_size, hidden2_size).astype(np.float32))

# Set the bais of the second layer - 50*1
# The values - 0.1
b2 = tf.Variable(tf.constant(0.1, shape=[hidden2_size]))

# Set the output of the second layer using ReLU
# Output size: [?,50]
z2 = tf.nn.relu(tf.matmul(z1,W2)+b2)

# Set the weight of the output layer - 50*25
# The values to be normalized [0.1,1]
W3 = tf.Variable(tf.zeros([hidden2_size, num_categories]))

# Set the bais of the output layer - 25*1
# The values - 0.1
b3 = tf.Variable(tf.constant(0.1, shape=[num_categories]))

# Set the output using SoftMax
# Output size: [?,25]
y = tf.nn.softmax(tf.matmul(z2, W3) + b3)
#y = 1 / (1.0 + tf.exp(-z2))

# Define the loss function - Cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# Define the update function - Gradient descent
update = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

# Define accuracy - Percentage of predictions did we get right
correct_prediction = tf.equal(tf.argmax(y, 1),tf.argmax(y_, 1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) 

In [201]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(0,1000):
    sess.run(update, feed_dict = {x:data_x, y_:data_y})
    #print(W1.eval(session=sess))
    if epoch%10 == 0:
        train_acc = sess.run(accuracy, feed_dict={x:data_x, y_:data_y})
        test_acc = sess.run(accuracy, feed_dict={x:x_test, y_:y_test})
        print ("epoch: %3d train_acc: %.3f test_acc: %.3f" % (epoch,train_acc, test_acc))

epoch:   0 train_acc: 0.047 test_acc: 0.020
epoch:  10 train_acc: 0.109 test_acc: 0.090
epoch:  20 train_acc: 0.142 test_acc: 0.147
epoch:  30 train_acc: 0.232 test_acc: 0.185
epoch:  40 train_acc: 0.275 test_acc: 0.261
epoch:  50 train_acc: 0.298 test_acc: 0.254
epoch:  60 train_acc: 0.297 test_acc: 0.229
epoch:  70 train_acc: 0.310 test_acc: 0.256
epoch:  80 train_acc: 0.330 test_acc: 0.270
epoch:  90 train_acc: 0.345 test_acc: 0.283
epoch: 100 train_acc: 0.360 test_acc: 0.312
epoch: 110 train_acc: 0.372 test_acc: 0.319
epoch: 120 train_acc: 0.395 test_acc: 0.342
epoch: 130 train_acc: 0.403 test_acc: 0.339
epoch: 140 train_acc: 0.421 test_acc: 0.358
epoch: 150 train_acc: 0.425 test_acc: 0.358
epoch: 160 train_acc: 0.440 test_acc: 0.370
epoch: 170 train_acc: 0.446 test_acc: 0.377
epoch: 180 train_acc: 0.455 test_acc: 0.381
epoch: 190 train_acc: 0.457 test_acc: 0.375
epoch: 200 train_acc: 0.463 test_acc: 0.384
epoch: 210 train_acc: 0.469 test_acc: 0.388
epoch: 220 train_acc: 0.475 test